## Урок 2. Веб запросы, работа с API, парсинг

1. Доработать приложение по поиску авиабилетов, чтобы оно возвращало билеты по названию города, а не по IATA коду. (У aviasales есть для этого дополнительное API) Пункт отправления и пункт назначения должны передаваться в качестве параметров. Сделать форматированный вывод, который содержит в себе пункт отправления, пункт назначения, дату вылета, цену билета (можно добавить еще другие параметры по желанию)

In [70]:
from pprint import pprint
import requests
import json

fromCity = input('Пункт отправления: ')
toCity = input('Пункт назначения: ')
search = f'Из {fromCity} в {toCity}'
service_iata = 'https://www.travelpayouts.com/widgets_suggest_params?q='
link_iata = f'{service_iata}{search}'
req_iata = requests.get(link_iata)
data_iata = json.loads(req_iata.text)
fromCity = data_iata['origin']['iata']
toCity = data_iata['destination']['iata']

service = 'http://min-prices.aviasales.ru/calendar_preload?'
link = f'{service}origin={fromCity}&destination={toCity}&one_way=true'
req = requests.get(link)
data = json.loads(req.text)
print('Дата вылета   Цена  Класс  Перес.  Дата/время изменения  Сайт')
for i in data['best_prices']:
    print(f"{i['depart_date']}   {i['value']}   {i['trip_class']}      {i['number_of_changes']}     {i['found_at']}  {i['gate']}")


Пункт отправления: Москва
Пункт назначения: Лондон
Дата вылета   Цена  Класс  Перес.  Дата/время изменения  Сайт
2019-10-01   2020.0   0      0     2019-08-16T16:15:44  OneTwoTrip
2019-12-22   3624.0   0      1     2019-08-11T19:14:11  Kiwi.com
2020-03-04   4192.0   0      1     2019-08-11T08:44:25  Trip.ru
2019-10-02   2921.0   0      0     2019-08-12T14:55:22  OneTwoTrip
2020-01-01   9097.0   0      1     2019-08-12T06:26:47  Trip.ru
2020-07-10   6090.0   0      1     2019-08-16T16:12:59  Trip.ru
2019-10-11   2921.0   0      0     2019-08-12T14:05:03  Wizz Air
2020-01-04   12969.0   0      1     2019-08-16T15:29:18.438128  AMADEUS
2020-02-07   4027.0   0      1     2019-08-10T19:06:16  Trip.ru
2020-03-21   4022.0   0      1     2019-08-12T08:07:37  Trip.ru
2019-10-18   2921.0   0      0     2019-08-12T11:43:23  Svyaznoy Travel
2019-09-28   5199.0   0      1     2019-08-11T11:14:26  Kiwi.com
2019-12-08   3286.0   0      0     2019-08-11T14:20:33  Wizz Air
2019-10-19   2190.0   0      

2. В приложении парсинга википедии получить первую ссылку из раздела "Ссылки" и вывести все значимые слова из неё. Результат записать в файл в форматированном виде.

In [210]:
from pprint import pprint
import requests
import re

def get_link(topic):
    if topic[0:4] != 'http':
        link='https://ru.wikipedia.org/wiki/'+topic.capitalize()
    else:
        link=topic
    return link

def get_topic_page(topic):
    link = get_link(topic)
    html = requests.get(link).text
    return html

def get_topic_text(topic):
    html_content = get_topic_page(topic)
    words = re.findall("[а-яА-Я]{3,}", html_content)
    #text = ' '.join(words)
    return words

def get_common_words(topic):
    words_list = get_topic_text(topic)
    rate={}
    for word in words_list:
        if word in rate:
            rate[word]+=1
        else:
            rate[word]=1
    rate_list = list(rate.items())
    rate_list.sort(key = lambda x: -x[1])
    return rate_list

def get_topic_link(topic):
    html_content = get_topic_page(topic)
    links = re.findall('"Ссылки"[\S\s]*?(?<=href=")(http:\/\/.*?(?="))',html_content)
    return links   

link1 = get_topic_link('Россия')
pprint(link1)
dict2 = get_common_words(str(link1))
pprint(dict2)

with open('dict2.txt', 'w', encoding='utf-8') as f:
    f.write('Частота  Слово\n')
    for i in dict2:
        f.write(f'   {i[1]}     {i[0]}\n')


['http://www.kremlin.ru']
[('название', 11),
 ('Недопустимое', 9),
 ('Служебная', 8),
 ('Википедия', 6),
 ('Перейти', 6),
 ('страницы', 5),
 ('страница', 5),
 ('для', 4),
 ('Список', 4),
 ('Поиск', 4),
 ('Википедии', 4),
 ('Заглавная', 3),
 ('можно', 3),
 ('страницу', 3),
 ('май', 2),
 ('странице', 2),
 ('Навигация', 2),
 ('системе', 2),
 ('тную', 2),
 ('запись', 2),
 ('это', 2),
 ('Искать', 2),
 ('Найти', 2),
 ('заглавную', 2),
 ('Статьи', 2),
 ('Сообщить', 2),
 ('ошибке', 2),
 ('этой', 2),
 ('где', 2),
 ('Форум', 2),
 ('страниц', 2),
 ('Версия', 2),
 ('печати', 2),
 ('Описание', 2),
 ('Отказ', 2),
 ('ответственности', 2),
 ('январь', 1),
 ('февраль', 1),
 ('март', 1),
 ('апрель', 1),
 ('июнь', 1),
 ('июль', 1),
 ('август', 1),
 ('сентябрь', 1),
 ('октябрь', 1),
 ('ноябрь', 1),
 ('декабрь', 1),
 ('янв', 1),
 ('фев', 1),
 ('мар', 1),
 ('апр', 1),
 ('июн', 1),
 ('июл', 1),
 ('авг', 1),
 ('сен', 1),
 ('окт', 1),
 ('ноя', 1),
 ('дек', 1),
 ('русский', 1),
 ('лента', 1),
 ('навигации', 1),

3. \* Научить приложение определять количество ссылок в статье (раздел Ссылки). Выполнить поиск слов в статьях по каждой ссылке и результаты записать в отдельные файлы.